## **TOPIC MODELLING AND RECOMENDATION**

In [1]:
## **LOAD PACKAGES**

import pandas as pd
import os
from datetime import datetime
import numpy as np

#Sometimes I will use R functions (those I believe are the best ones)
# Load in the r magic
%load_ext rpy2.ipython

In [2]:
#load treated and aggregated datas
df = pd.read_csv('data/treated_data.csv') 
print(df.shape)

(9860, 14)


In [3]:
df.dtypes

Page Title            object
Source                object
Country               object
Device Category       object
Sessions               int64
Avg. Time on Page      int64
Bounce Rate          float64
Page Views             int64
Unique Page Views      int64
Users                  int64
url_title             object
content               object
date                  object
title                 object
dtype: object

In [4]:
## We will keep only topics in spanish, but filtering by country does not work
#df = df[df['Country'] == 'spain']

In [5]:
df.head()

Page Title        Source  \
0  financial text classification: an analysis of ...      (direct)   
1  financial text classification: an analysis of ...      (direct)   
2  financial text classification: an analysis of ...          t.co   
3  financial text classification: an analysis of ...  linkedin.com   
4  financial text classification: an analysis of ...        google   

         Country Device Category  Sessions  Avg. Time on Page  Bounce Rate  \
0          spain          mobile        47                339       0.8511   
1          spain         desktop        19                698       0.7368   
2          spain          mobile        14                  0       1.0000   
3          spain         desktop        13                685       0.8462   
4  united states         desktop         9                192       0.8889   

   Page Views  Unique Page Views  Users url_title  \
0          64                 50     28    4062-2   
1          30                 25     20    4062-2   
2          14                 14      5    4062-2   
3          16                 14     12    4062-2   
4          13                 12      8    4062-2   

                                             content        date  \
0  \ndurante los últimos dos veranos he trabajado...  2018-06-11   
1  \ndurante los últimos dos veranos he trabajado...  2018-06-11   
2  \ndurante los últimos dos veranos he trabajado...  2018-06-11   
3  \ndurante los últimos dos veranos he trabajado...  2018-06-11   
4  \ndurante los últimos dos veranos he trabajado...  2018-06-11   

                                               title  
0  clasificación de texto financiero: un análisis...  
1  clasificación de texto financiero: un análisis...  
2  clasificación de texto financiero: un análisis...  
3  clasificación de texto financiero: un análisis...  
4  clasificación de texto financiero: un análisis...

In [6]:
# there are duplicated information for topic modelling, so it would be nice to eliminate duplicated for this.
columns_val = ['content', 'url_title', 'Country']
df_topic = df[columns_val]
print(df_topic.shape)
df_topic = df_topic.drop_duplicates(subset ='content')
print(df_topic.shape)

(9860, 3)
(105, 3)


In [7]:
df_topic

content  \
0     \ndurante los últimos dos veranos he trabajado...   
113   \nla economía global digital continuará expand...   
339   \nhay increíbles introducciones, cursos y entr...   
521   \nmaximizar las ganancias es una premisa que s...   
665   \nmuchas consideraciones juegan un papel impor...   
...                                                 ...   
9515  \nin our recent collaboration with united nati...   
9543  \nla huella digital derivada de nuestra activi...   
9647  \nno hace mucho tiempo, mucha gente todavía se...   
9846  \nwe are proud to be part of bbva’s technologi...   
9849  \nadmittedly, i always find it difficult to na...   

                                              url_title               Country  
0                                                4062-2                 spain  
113         7-challenges-for-the-global-digital-economy                 india  
339               a-weird-introduction-to-deep-learning         united states  
521   adding-fairness-to-dynamic-pricing-with-reinfo...                 spain  
665   an-international-football-story-how-to-analyze...                 spain  
...                                                 ...                   ...  
9515                           what-does-wind-look-like             hong kong  
9543                 what-does-your-bank-know-about-you               uruguay  
9647         what-will-the-bank-of-the-future-look-like                 spain  
9846  part-of-bbvas-technological-progress-and-trans...         united states  
9849                        scaling-to-lots-of-problems  united arab emirates  

[105 rows x 3 columns]

## **DETECT LANGUAGE**

In [8]:
from langdetect import detect
df_topic['language'] = df_topic['content'].apply(detect)

In [9]:
df_topic.tail()

content  \
9515  \nin our recent collaboration with united nati...   
9543  \nla huella digital derivada de nuestra activi...   
9647  \nno hace mucho tiempo, mucha gente todavía se...   
9846  \nwe are proud to be part of bbva’s technologi...   
9849  \nadmittedly, i always find it difficult to na...   

                                              url_title               Country  \
9515                           what-does-wind-look-like             hong kong   
9543                 what-does-your-bank-know-about-you               uruguay   
9647         what-will-the-bank-of-the-future-look-like                 spain   
9846  part-of-bbvas-technological-progress-and-trans...         united states   
9849                        scaling-to-lots-of-problems  united arab emirates   

     language  
9515       en  
9543       es  
9647       es  
9846       en  
9849       en

In [10]:
#TextBlob(df.iloc[1,1]).detect_language()

In [11]:
#df_topic["content"].apply(language_txt)

In [12]:
df_topic = df_topic[df_topic["language"] != 'en']
df_topic.tail()

content  \
9243  \nlas aplicaciones de la física estadística y ...   
9348  \nlas ciudades son hardware físico que sirve d...   
9395  \na finales del año pasado, urban discovery mo...   
9543  \nla huella digital derivada de nuestra activi...   
9647  \nno hace mucho tiempo, mucha gente todavía se...   

                                              url_title  Country language  
9243           untangle-cascade-effects-complex-systems    spain       es  
9348  urban-analytics-to-measure-the-economic-impact...    spain       es  
9395         urban-discovery-a-vision-to-rethink-cities    spain       es  
9543                 what-does-your-bank-know-about-you  uruguay       es  
9647         what-will-the-bank-of-the-future-look-like    spain       es

## **Data treatment**

## **STOPWORDS**

In [13]:
from nltk.corpus import stopwords
stop = stopwords.words('spanish')
# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
df_topic['content'] = df_topic['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

/home/erikapat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## **LEMMATIZATION**

In [14]:
## lematization
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
lemma=WordNetLemmatizer()
token=nltk.data.load('tokenizers/punkt/spanish.pickle')

def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w,'v')
        listLemma.append(x)
    return text
df_topic['content'] = df_topic['content'].map(lambda x : lemitizeWords(x))

/home/erikapat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [15]:
df_topic

content  \
0     últimos dos veranos trabajado becario bbva dat...   
113   economía global digital continuará expandiéndo...   
339   increíbles introducciones, cursos entradas blo...   
521   maximizar ganancias premisa siempre presente m...   
665   muchas consideraciones juegan papel importante...   
...                                                 ...   
9243  aplicaciones física estadística teoría grafos ...   
9348  ciudades hardware físico sirve soporte múltipl...   
9395  finales año pasado, urban discovery mostró nue...   
9543  huella digital derivada actividad bancaria (tr...   
9647  hace tiempo, mucha gente todavía presentaba of...   

                                              url_title        Country  \
0                                                4062-2          spain   
113         7-challenges-for-the-global-digital-economy          india   
339               a-weird-introduction-to-deep-learning  united states   
521   adding-fairness-to-dynamic-pricing-with-reinfo...          spain   
665   an-international-football-story-how-to-analyze...          spain   
...                                                 ...            ...   
9243           untangle-cascade-effects-complex-systems          spain   
9348  urban-analytics-to-measure-the-economic-impact...          spain   
9395         urban-discovery-a-vision-to-rethink-cities          spain   
9543                 what-does-your-bank-know-about-you        uruguay   
9647         what-will-the-bank-of-the-future-look-like          spain   

     language  
0          es  
113        es  
339        es  
521        es  
665        es  
...       ...  
9243       es  
9348       es  
9395       es  
9543       es  
9647       es  

[67 rows x 4 columns]

## **TOPIC MODELLING**

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=None)
dtm = tfidf.fit_transform(df_topic['content'])

In [17]:
dtm

<67x3514 sparse matrix of type '<class 'numpy.float64'>'
	with 18082 stored elements in Compressed Sparse Row format>

## **NMF: Non-Negative Matrix Factorization (NMF)**

In [18]:
from sklearn.decomposition import NMF
nmf_model = NMF(n_components=7,random_state=42)
# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=7, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [19]:
len(tfidf.get_feature_names())

3514

In [20]:
import random
for i in range(10):
    random_word_id = random.randint(0,len(tfidf.get_feature_names()))
    print(tfidf.get_feature_names()[random_word_id])

orientación
estadística
importantes
básicas
turismo
modelar
tarea
navegar
efectiva
72


In [21]:
for i in range(10):
    random_word_id = random.randint(0,len(tfidf.get_feature_names()))
    print(tfidf.get_feature_names()[random_word_id])

aprovechando
recibir
implementar
velocidad
trabajos
línea
esto
privacidad
automatically
bares


In [22]:
len(nmf_model.components_)

7

In [23]:
nmf_model.components_

array([[0.00224602, 0.0159113 , 0.00387331, ..., 0.00246183, 0.02449095,
        0.01728084],
       [0.        , 0.        , 0.00411597, ..., 0.        , 0.00822863,
        0.01202506],
       [0.05627031, 0.01666374, 0.00892857, ..., 0.        , 0.        ,
        0.00289509],
       ...,
       [0.02557899, 0.03069361, 0.01873116, ..., 0.00423099, 0.03194008,
        0.01042267],
       [0.01235411, 0.        , 0.00136871, ..., 0.        , 0.00332559,
        0.0133641 ],
       [0.00358471, 0.        , 0.00574222, ..., 0.0052786 , 0.02096017,
        0.        ]])

In [24]:
len(nmf_model.components_[0])

3514

In [25]:
single_topic = nmf_model.components_[0]

In [26]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([1756, 2124,  858, ...,  929,  263, 1962])

In [27]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([ 875,  699, 2262,  902, 2908, 2858, 2030,  929,  263, 1962])

In [28]:
top_word_indices = single_topic.argsort()[-10:]

In [29]:
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

cursos
concepto
neuronales
datos
representación
redes
machine
deep
aprendizaje
learning


In [30]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['profundo', 'charla', 'algoritmos', 'modelos', 'automático', 'cursos', 'concepto', 'neuronales', 'datos', 'representación', 'redes', 'machine', 'deep', 'aprendizaje', 'learning']


THE TOP 15 WORDS FOR TOPIC #1
['nuevos', 'talento', 'science', 'negocio', 'cómo', 'personas', 'información', 'entrevista', 'clientes', 'empresas', 'personales', 'big', 'bbva', 'data', 'datos']


THE TOP 15 WORDS FOR TOPIC #2
['consumo', 'atributos', 'urban', 'barrios', 'áreas', 'zona', 'méxico', 'actividad', 'datos', 'barcelona', 'comercial', 'madrid', 'zonas', 'ciudades', 'ciudad']


THE TOP 15 WORDS FOR TOPIC #3
['on', 'machine', 'microsoft', 'deep', 'tensorflow', 'gpu', 'sql', 'summit', 'comunidad', 'datos', 'streaming', 'learning', 'docker', 'apache', 'spark']


THE TOP 15 WORDS FOR TOPIC #4
['pago', 'ingresos', 'bconomy', 'comentarios', 'transacción', 'predicción', 'financiera', 'incertidumbre', 'datos', 'motor', 'transacciones', 'gastos', 'cliente', 'bbva', 'clientes']




In [31]:
topic_results = nmf_model.transform(dtm)

In [32]:
topic_results.shape

(67, 7)

In [33]:
df_topic['Topic'] = topic_results.argmax(axis=1)

/home/erikapat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## **UNDERSTAND THE TOPICS**

In [34]:
pd.set_option('display.max_colwidth', 199)  # or 199
df_topic = df_topic.sort_values(by=['Topic'])
df =df_topic[['url_title', 'content', 'Topic']]

In [35]:
print(df[df['Topic'] == 0].shape)
df[df['Topic'] == 0].head()

(10, 3)


url_title  \
6722                         saw-liked-2017-part-3   
5049          not-deep-learning-learning-represent   
339          a-weird-introduction-to-deep-learning   
6789                                saw-liked-2017   
6930  saw-open-data-science-conference-europe-2017   

                                                                                                                                                                                                     content  \
6722  campo amplio machine learning: análisis redes nuevas interacciones máquinas interacciones máquinas humanos — iskra velitchkova ciencia, tradicionalmente vivido cómoda complejidad, exige, parte so...   
5049  recientemente, creado grupo trabajo dedicado deep learning (aprendizaje profundo). grupos trabajo ofrecen oportunidades compartir internamente ideas, conceptos, recursos, código, etc. además, pre...   
339   increíbles introducciones, cursos entradas blogs deep learning. voy nombrar secciones recursos, clase diferente introducción: introducción peculiar. ¿pero peculiar? tal vez sigue estructura “norm...   
6789  bbva data & analytics encanta compartir artículos, blogs noticias inteligencia artificial (ai) aprendizaje automático (ml) mantener día compromiso pasión utilización aprendizaje automático campo ...   
6930  odsc londres 2017 mostró asombrosa variedad herramientas, librerías, notebooks aplicaciones ciencia datos. 75 ponentes, 1.500 asistentes literalmente espacio más. importa si prefieres python, r i...   

      Topic  
6722      0  
5049      0  
339       0  
6789      0  
6930      0

In [36]:
print(df[df['Topic'] == 1].shape)
df[df['Topic'] == 1].head()

(19, 3)


url_title  \
8653                                                     training-an-essential-perk-to-retain-a-data-scientist-profile   
3692  finding-a-common-vision-design-fiction-to-establish-a-dialogue-among-business-data-science-and-design-in-finance   
4418                                                  innovation-will-become-customer-focused-participation-enter-2018   
6288                                                     research-and-innovation-what-the-future-looks-like-for-the-eu   
9543                                                                                what-does-your-bank-know-about-you   

                                                                                                                                                                                                     content  \
8653  primer hito bbva data & analytics alcanzó 2011, unidad innovación bbva creó pequeño equipo investigación, comprometido uso fuentes datos existentes resolver retos negocio. varias colaboraciones e...   
3692  pasado hablado cómo práctica design fiction bbva d&a ayudado alinear romper fronteras equipos científicos datos stakeholders, torno potenciales analítica avanzada aplicada ámbitos desarrollo prod...   
4418  enter 2018, organizado anualmente federación internacional tecnologías información, viajes turismo (ifitt), lugar última semana enero jönköping (una encantadora ciudad sueca orillas lago vättern)...   
6288  ¿cuáles tecnologías perfilarán futuro unión europea?¿quiénes jugadores implicados creación innovaciones llegar? propósito dar respuesta preguntas analizado fuentes datos abiertos comisión europea...   
9543  huella digital derivada actividad bancaria (transferencias realizadas, movimientos tarjetas, etc.), supone fuente información rica actualización permanente, permite banco desarrollar productos se...   

      Topic  
8653      1  
3692      1  
4418      1  
6288      1  
9543      1

In [37]:
print(df[df['Topic'] == 2].shape)
df[df['Topic'] == 2].head()

(8, 3)


url_title  \
9243                           untangle-cascade-effects-complex-systems   
9348  urban-analytics-to-measure-the-economic-impact-of-traffic-calming   
9218    understanding-economic-redistribution-with-credit-card-payments   
9395                         urban-discovery-a-vision-to-rethink-cities   
6004                              rediscovering-cities-credit-card-data   

                                                                                                                                                                                                     content  \
9243  aplicaciones física estadística teoría grafos fenómenos sociales ayudando descubrir patrones comportamiento inesperados permiten reaccionar manera efectiva eventos desestabilizadores redes comple...   
9348  ciudades hardware físico sirve soporte múltiples dinámicas superpuestas, resultado superposición sistema extremadamente complejo. cualquier cambio condiciones produce efectos positivos negativos ...   
9218  antecedentes ciudades contienen barrios desempeñan económicamente mejor otros. casos diferencias extremas, ocurre río janeiro, ingreso promedio barrios ricos copacabana ipanema puede ser 100% alt...   
9395  finales año pasado, urban discovery mostró nueva forma representar ciudades basada comportamiento compra. enfoque permitió bbva data & analytics examinar madrid, barcelona ciudad méxico bajo nuev...   
6004  — metodología proceso analítico divisiones oficiales ciudades, distritos barrios, siendo utilizadas sólo gobernar ciudades sino describir comportamiento ciudadanos. hecho, estadísticas oficiales ...   

      Topic  
9243      2  
9348      2  
9218      2  
9395      2  
6004      2

In [38]:
print(df[df['Topic'] == 3].shape)
df[df['Topic'] == 3].head()

(3, 3)


url_title  \
7068                                                                 saw-spark-summit-2017-spark-embraces-deep-learning   
2804                                                                    environment-spark-less-30-seconds-thanks-docker   
7142  saw-spark-summit-europe-2017-spark-libraries-real-machine-learning-systems-production-reality-deep-learning-spark   

                                                                                                                                                                                                     content  \
7068  conferencia spark summit 2017, lugar san francisco pasado mes junio, atrajo 3000 miembros distintos sectores participaron 170 sesiones discutir tendencias construirán productos basados datos futu...   
2804  seguro alguna vez querido “cacharrear” alguna tecnología, afán aprender seguro visto mermado dificultad instalar necesario sistema operativo. artículo explicaremos cómo empezar jugar spark import...   
7142  hace mes lugar último spark summit europe, evento grande mundo comunidad apache spark™, participaron 1200 desarrolladores, ingenieros, científicos datos, investigadores profesionales negocio. jos...   

      Topic  
7068      3  
2804      3  
7142      3

In [39]:
print(df[df['Topic'] == 4].shape)
df[df['Topic'] == 4].head()

(13, 3)


url_title  \
6110                            reinforcement-learning-for-pricing-strategy-optimization   
0                                                                                 4062-2   
5154                                   one-click-closer-to-an-ai-powered-account-manager   
4719  learning-cardiac-arrhythmias-improve-customer-satisfaction-working-imbalanced-data   
4592                                        kitties-helped-bbva-detect-credit-card-fraud   

                                                                                                                                                                                                     content  \
6110  darle vuelta tortilla movimiento complejo humanos necesitan aprender través experimentación repetición. entonces, ¿cómo podría robot aprender dar vuelta tortilla? imaginemos brazo robotizado capa...   
0     últimos dos veranos trabajado becario bbva data & analytics, tiempo desarrollado trabajo gustaría explorar artículo. sigue resumen técnico aprendizajes adquiridos trabajando embedding palabras ca...   
5154  mientras preparas café mañana, corres junto río justo subirte coche. ahora puedes completar transferencia pago, administrar salud cuenta perder vista rutina, llenar tediosa lista números formular...   
4719  antecedentes satisfacción cliente indicador complejo. puede cambiar después cada interacción, puede ser influenciada factores externos puede conducir múltiples resultados. insatisfacción acumulat...   
4592  grupo científicos datos bbva data & analytics desarrollado modelo aprendizaje profundo utiliza novedoso enfoque detección fraudes tarjetas crédito. metodología sido aplicada científicos datos bbv...   

      Topic  
6110      4  
0         4  
5154      4  
4719      4  
4592      4

In [40]:
print(df[df['Topic'] == 5].shape)
df[df['Topic'] == 5].head(7)

(10, 3)


url_title  \
8435                               the-most-important-developments-in-data-science-of-2018   
7322  stephen-hawking-genius-saw-potential-responsible-development-artificial-intelligence   
4899                                                                   nips-chronicles-saw   
4753              lifting-the-veil-of-interpretability-in-a-world-controlled-by-algorithms   
3580                            fairness-by-design-in-machine-learning-is-going-mainstream   
3122                  eurovision-2018-just-another-excuse-to-demonstrate-the-value-of-data   
1577                            bringing-the-artificial-intelligence-to-its-true-potential   

                                                                                                                                                                                                     content  \
8435  año punto dar finalizado sido importantes cuanto desarrollo tecnologías aprendizaje autónomo (machine learning). 2018, gran parte debate centrado cómo seguir perfeccionando tecnologías pasar simp...   
7322  stephen hawking (1942-2018) dejó mundo dedicarse estrellas pi day, celebrado matemáticos físicos mundo. leyeron “una breve historia tiempo” decidieron explorar mundo través números ojos científic...   
4899  puede convertido tradición: diciembre momento muestran frutos maduros 12 meses investigación global aprendizaje automático definen perfilan tendencias futuras parece haberse convertido reunión im...   
4753  cada vez rápida adopción inteligencia artificial (ai) aprendizaje autónomo (machine learning) importantes ámbitos sociedad, falta transparencia torno procesos convertido preocupación perceptible ...   
3580  consideraciones equidad “fairness” desarrollo soluciones basadas machine learning (aprendizaje automático) cobrando fuerza aspecto clave inteligencia artificial modelación comportamientos sociale...   
3122  datos cada vez específicos, dimensión profunda mayores niveles riqueza individuos u organizaciones puedan sacar conclusiones. datos especialmente granulares estructurados afectan operaciones bási...   
1577  término inteligencia artificial acuñado john mccarthy conferencia darmouth 1956 alcanzado grandes logros ámbito: enseñado máquinas ver, reconociendo imágenes incluso texto. enseñado leer, escucha...   

      Topic  
8435      5  
7322      5  
4899      5  
4753      5  
3580      5  
3122      5  
1577      5

In [41]:
print(df[df['Topic'] == 6].shape)
df[df['Topic'] == 6].head(7)

(4, 3)


url_title  \
5829                                   recsys-debate-widen-world-find-new-ones   
5541                                   reading-list-machine-learning-designers   
2165  cost-effective-scalable-collaborative-filtering-based-recommender-system   
6997                                                saw-recsys-2017-conference   

                                                                                                                                                                                                     content  \
5829  bbva data & analytics creemos futuro tecnología debe ser continuamente cuestionado, debatido considerado diferentes puntos vista. filosofía, organizamos semana encuentro convencional artistas, ju...   
5541  bbva data & analytics continuamente colaboramos equipos diseño bbva crear experiencias usuario basadas técnicas aprendizaje automático (por ejemplo, modelos predictivos, sistemas recomendación). ...   
2165  pasado jueves (26 septiembre) oportunidad presentar colaboración bbva data & analytics beeva theatre of partners aws summit madrid 2017. conferencia, presentamos enfoque rentable sistemas recomen...   
6997  recsys, lugar pasado 27 agosto 2017, mayores jornadas académicas sistemas recomendación (recsys) alcanzado año undécima edición récord histórico asistentes (627), demuestra creciente importancia ...   

      Topic  
5829      6  
5541      6  
2165      6  
6997      6

In [42]:
# 0: aprendizaje data science machine learning
# 1: digital change in companies/companies and machine learning
# 2: SMART CITIES
# 3: TOOLS FOR MACHINE LEARNING/TECH
# 4: PERSONALIZATION
# 5: FUTURE/ADVANCE/THE FUTURE AND PAST OF SCIENCE
# 6: RESEARCH

In [43]:
df.head()

url_title  \
6722                         saw-liked-2017-part-3   
5049          not-deep-learning-learning-represent   
339          a-weird-introduction-to-deep-learning   
6789                                saw-liked-2017   
6930  saw-open-data-science-conference-europe-2017   

                                                                                                                                                                                                     content  \
6722  campo amplio machine learning: análisis redes nuevas interacciones máquinas interacciones máquinas humanos — iskra velitchkova ciencia, tradicionalmente vivido cómoda complejidad, exige, parte so...   
5049  recientemente, creado grupo trabajo dedicado deep learning (aprendizaje profundo). grupos trabajo ofrecen oportunidades compartir internamente ideas, conceptos, recursos, código, etc. además, pre...   
339   increíbles introducciones, cursos entradas blogs deep learning. voy nombrar secciones recursos, clase diferente introducción: introducción peculiar. ¿pero peculiar? tal vez sigue estructura “norm...   
6789  bbva data & analytics encanta compartir artículos, blogs noticias inteligencia artificial (ai) aprendizaje automático (ml) mantener día compromiso pasión utilización aprendizaje automático campo ...   
6930  odsc londres 2017 mostró asombrosa variedad herramientas, librerías, notebooks aplicaciones ciencia datos. 75 ponentes, 1.500 asistentes literalmente espacio más. importa si prefieres python, r i...   

      Topic  
6722      0  
5049      0  
339       0  
6789      0  
6930      0

In [44]:
dtm

<67x3514 sparse matrix of type '<class 'numpy.float64'>'
	with 18082 stored elements in Compressed Sparse Row format>

In [45]:
df = df[['url_title', 'Topic']]
df.to_csv(r'data/topic_data.csv', index = False)

In [46]:
dth =df_topic['content']

## **RECOMEND SIMILAR ARTICLES**

In [47]:
nmf_features = nmf_model.fit_transform(dtm)
from sklearn.preprocessing import normalize
norm_features = normalize(nmf_features)
df = pd.DataFrame(norm_features)
current_article = df.iloc[1, :]
similarities = df.dot(current_article)

In [48]:
print(similarities.nlargest()) 

1     1.000000
34    0.989774
44    0.981976
8     0.970335
23    0.969918
dtype: float64


In [49]:
type(df_topic)

pandas.core.frame.DataFrame

In [50]:
d_t = pd.DataFrame(df_topic)
d_t = d_t[['content', 'Topic']]
d_t.iloc[1,:]

content    recientemente, creado grupo trabajo dedicado deep learning (aprendizaje profundo). grupos trabajo ofrecen oportunidades compartir internamente ideas, conceptos, recursos, código, etc. además, pre...
Topic                                                                                                                                                                                                           0
Name: 5049, dtype: object

## **EX: RECOMEND SIMILAR ARTICLES**

In [51]:
#similar articles are: the same article and other 3.

In [52]:
d_t.iloc[list(pd.DataFrame(similarities.nlargest()).index),:]

content  \
5049  recientemente, creado grupo trabajo dedicado deep learning (aprendizaje profundo). grupos trabajo ofrecen oportunidades compartir internamente ideas, conceptos, recursos, código, etc. además, pre...   
5913  — lectura resultados perspectiva urbanística antecedentes artículo anterior descrito metodología, modelos herramientas analíticas representación empleados proyecto urban discovery, desarrollado j...   
4592  grupo científicos datos bbva data & analytics desarrollado modelo aprendizaje profundo utiliza novedoso enfoque detección fraudes tarjetas crédito. metodología sido aplicada científicos datos bbv...   
6531  noviembre 2017, nueva orleans (louisiana), acogió conferencia importante mundo minería datos. 17ª edición evento presentó nuevos trabajos analítica gráfica, patrones series temporales sistemas re...   
1476  jon ander beracoechea — co-ceo bbva data & analytics jon doctorado ingeniería eléctrica publicado múltiples trabajos campo procesamiento adaptativo señales. hace 9 años aplica técnicas avanzadas ...   

      Topic  
5049      0  
5913      2  
4592      4  
6531      0  
1476      1